In [1]:
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm
import IPython.display as display
import copy
import seaborn as sns

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/1k/27mkp8bj3ps60c3nmr7rbqzh0000gn/T/ipykernel_12930/3054088700.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch shou

In [2]:


def merge_csv_files(directory):
    # Get a list of all the csv files
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Initialize an empty list to hold dataframes
    dfs = []

    # Loop through csv files, read each into a dataframe, and append to the list
    for file in csv_files:
        # Extract date from filename, assuming the date is in format 'traffic_flow_YYYY_MM_DD'
        date_str = file.split('.')[0].split('_')[-3:]  # This gives ['YYYY', 'MM', 'DD']
        date = datetime.strptime('_'.join(date_str), '%Y_%m_%d').date()

        df = pd.read_csv(os.path.join(directory, file))

        # Add date as a new column
        df['date'] = date.strftime('%m/%d/%y')

        dfs.append(df)

    # Concatenate all dataframes in the list into one dataframe
    merged_df = pd.concat(dfs, ignore_index=True).drop_duplicates()

    # Return the merged dataframe
    return merged_df

traffic_flows = merge_csv_files(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

lsoa = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/london_LSOA/london_LSOA.shp'
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'
inoutter = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/lp-consultation-oct-2009-inner-outer-london-shp/lp-consultation-oct-2009-inner-outer-london.shp'
tube_line = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_lines.json'
tube_station = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_stations.json'

inoutter = gpd.read_file(inoutter)
inoutter.to_crs(epsg=27700, inplace=True)

# tube_station = gpd.read_file(tube_station)
# tube_station.to_crs(epsg=27700, inplace=True)
# tube_station = gpd.sjoin(tube_station, inoutter, op='within')

tube_line = gpd.read_file(tube_line)
tube_line.to_crs(epsg=27700, inplace=True)
tube_line = gpd.sjoin(tube_line, inoutter, op='within')

lsoa = gpd.read_file(lsoa, crs={'init': 'epsg:27700'})
road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()
traffic_flows['total'] = traffic_flows['bus'] + traffic_flows['car'] + traffic_flows['cycle'] + traffic_flows[
    'walks'] + traffic_flows['stationary']

flows = pd.merge(
    road_network[['toid', 'roadclassi', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg'
                  ]],
    traffic_flows, left_on='toid', right_on='toid', how='right')

flows['classification'] = flows['roadclassi'].replace(
    {'Unknown': 'Local Road', 'Not Classified': 'Local Road', 'Unclassified': 'Local Road',
     'Classified Unnumbered': 'Local Road'})

flows.drop(columns=['roadclassi'], inplace=True)

stage_date = ['03/01/22', '02/22/22', '03/08/22']
flows = flows.loc[flows['date'].isin(stage_date)]

# label the regional level
flows = gpd.sjoin(flows, inoutter, how='inner', predicate='within')
flows = flows.drop(columns=['index_right', 'Source', 'Area_Ha', 'Shape_Leng', 'Shape_Area'])
flows.reset_index(drop=True, inplace=True)

# convert the dataframe
flows = pd.melt(flows,
                id_vars=['toid', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
                         'date', 'Boundary'], var_name='mode', value_name='flow')

flows = pd.pivot_table(flows,
                       index=['toid', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
                              'date', 'Boundary', 'mode'], columns='date', values='flow', aggfunc='first').reset_index()

flows.drop(columns=['date'], inplace=True)
flows = flows.groupby(
    ['toid', 'mode', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
     'Boundary'], as_index=False).agg(
    {'03/01/22': 'first', '02/22/22': 'first', '03/08/22': 'first'})

# calculate the impact and recovery flows for one strike
flows['impact_flow'] = flows['03/01/22'] - flows['02/22/22']
flows['recovery_flow'] = flows['03/08/22'] - flows['03/01/22']

All = flows.copy()



date,toid,mode,classification,geometry,Boundary,cycle_lane,bus_lane,boroughs,03/01/22,02/22/22,03/08/22,impact_flow,recovery_flow,impact_rate,recovery_rate
0,osgb4000000027865921,bus,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,16,11,12,5,-4,0.4545,-0.2500
1,osgb4000000027865921,car,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1041,1100,1081,-59,40,-0.0536,0.0384
2,osgb4000000027865921,cycle,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,14,4,7,10,-7,2.5000,-0.5000
3,osgb4000000027865921,stationary,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,2,0,1,2,-1,0.0000,-0.5000
4,osgb4000000027865921,total,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1095,1151,1122,-56,27,-0.0487,0.0247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742635,osgb5000005242182149,car,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,33,54,51,-21,18,-0.3889,0.5455
1742636,osgb5000005242182149,cycle,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,2,1,1,1,-1,1.0000,-0.5000
1742637,osgb5000005242182149,stationary,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,3,4,5,-1,2,-0.2500,0.6667
1742638,osgb5000005242182149,total,Local Road,"LINESTRING (543548.945 179236.408, 543554.000 ...",Inner London,n,n,Greenwich,50,74,70,-24,20,-0.3243,0.4000


In [3]:
flows

date,toid,mode,classification,geometry,Boundary,cycle_lane,bus_lane,boroughs,03/01/22,02/22/22,03/08/22,impact_flow,recovery_flow,impact_rate,recovery_rate
0,osgb4000000027865921,total,Motorway,"LINESTRING (531539.442 200769.874, 531592.988 ...",Outer London,n,n,Enfield,1095,1151,1122,-56,27,-0.0487,0.0247
1,osgb4000000027865922,total,Motorway,"LINESTRING (531539.564 200789.167, 531577.468 ...",Outer London,n,n,Enfield,103,105,83,-2,-20,-0.0190,-0.1942
2,osgb4000000027866337,total,Motorway,"LINESTRING (530200.000 200559.000, 530212.000 ...",Outer London,n,n,Enfield,1231,1166,1247,65,16,0.0557,0.0130
3,osgb4000000027866338,total,Motorway,"LINESTRING (530189.000 200574.000, 530197.000 ...",Outer London,n,n,Enfield,1057,1128,1158,-71,101,-0.0629,0.0956
4,osgb4000000027869139,total,Local Road,"LINESTRING (530607.635 200580.747, 530609.796 ...",Outer London,n,n,Enfield,726,697,811,29,85,0.0416,0.1171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290435,osgb5000005242103810,total,Local Road,"LINESTRING (512458.577 178775.551, 512466.653 ...",Outer London,n,n,Ealing,6,6,11,0,5,0.0000,0.8333
290436,osgb5000005242113486,total,Local Road,"LINESTRING (544702.157 173400.459, 544700.305 ...",Inner London,n,n,Greenwich,2,0,0,2,-2,0.0000,-1.0000
290437,osgb5000005242156416,total,Local Road,"LINESTRING (522940.956 169325.977, 522936.738 ...",Outer London,n,n,Merton,79,63,69,16,-10,0.2540,-0.1266
290438,osgb5000005242156433,total,Strategic Road,"LINESTRING (522940.956 169325.977, 522915.000 ...",Outer London,n,n,Merton,306,371,321,-65,15,-0.1752,0.0490


In [4]:
fig, ax = plt.subplots()

cycle_lane = gpd.read_file('/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/CycleRoutes.json',crs={'init': 'epsg:4326'})
cycle_lane.to_crs(epsg=27700, inplace=True)
road_network.plot(color='lightgrey', ax=ax)
cycle_lane.plot(ax = ax, color = 'red')
plt.axis('off')
plt.show()

,Total Flows,mode,classification,Boundary,boroughs,03/01/22,02/22/22,03/08/22,cycle_lane,bus_lane,impact_flow,recovery_flow,impact_rate,recovery_rate
0,Total Flows,Bus,Local Road,Inner London,Camden,0.010747,0.022021,0.022628,N,N,0.785132,0.288018,0.166167,0.901885
1,Total Flows,Bus,Local Road,Inner London,City Of London,0.005768,0.009165,0.010165,N,N,0.822392,0.245652,0.214267,0.743829
2,Total Flows,Bus,Local Road,Inner London,Greenwich,0.013020,0.014118,0.013313,N,N,0.834164,0.220209,0.314000,0.402842
3,Total Flows,Bus,Local Road,Inner London,Hackney,0.008178,0.008614,0.010117,N,N,0.836835,0.230842,0.323233,0.501770
4,Total Flows,Bus,Local Road,Inner London,Hammersmith And Fulham,0.007074,0.007902,0.008053,N,N,0.834845,0.225510,0.304800,0.456276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Total Flows,Walks,Strategic Road,Outer London,Merton,0.041204,0.042459,0.043669,N,N,0.836309,0.226583,0.330433,0.420015
452,Total Flows,Walks,Strategic Road,Outer London,Redbridge,0.035988,0.040284,0.038932,N,N,0.821186,0.230525,0.304167,0.430157
453,Total Flows,Walks,Strategic Road,Outer London,Richmond Upon Thames,0.044425,0.049097,0.049141,N,N,0.820258,0.238942,0.308100,0.441351
454,Total Flows,Walks,Strategic Road,Outer London,Sutton,0.036153,0.038690,0.037542,N,N,0.829642,0.221453,0.318167,0.410161


/var/folders/1k/27mkp8bj3ps60c3nmr7rbqzh0000gn/T/ipykernel_12930/111802457.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(


,Bus/impact_rate,Car/impact_rate,Cycle/impact_rate,Stationary/impact_rate,Total/impact_rate,Walks/impact_rate,Bus/recovery_rate,Car/recovery_rate,Cycle/recovery_rate,Stationary/recovery_rate,Total/recovery_rate,Walks/recovery_rate
classification,,,,,,,,,,,,
Local Road,-0.110543,-0.049411,0.142011,-0.024463,-0.047426,-0.048226,0.187566,0.055089,-0.100137,0.046597,0.049586,0.047691
Motorway,0.091425,0.052513,0.473912,-0.023838,0.058537,0.144762,0.242013,-0.032563,-0.261012,-0.047137,-0.028125,-0.096825
Strategic Road,-0.098118,-0.062379,0.186209,0.008421,-0.056067,-0.070236,0.180406,0.052939,-0.115788,-0.029409,0.041812,0.039664
